<a href="https://colab.research.google.com/github/bwithhkj/Music-Recommender-System/blob/main/Song_recommendation_final_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MUSIC RECOMMENDATION SYSTEM

In [180]:
 import pandas as pd

In [181]:
import numpy as np

In [182]:
 
from sklearn.model_selection import train_test_split
import time
from sklearn.externals import joblib

Imported user data and song metadata

In [183]:
triplets_file =  'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'
song_df_1 = pd.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']
#Read song metadatasong_df_2 =  pd.read_csv(songs_metadata_file)
#Merge the two dataframes above to create input dataframe for recommender systemssong_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")
#EXPLORE DATA (This show how many times a user )song_df.head()
#Length of the datasetlen(song_df)

In [184]:
 song_df_2 =  pd.read_csv(songs_metadata_file)

In [185]:
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left")

In [186]:
 song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [187]:
 len(song_df)

2000000

In [188]:
 song_df = song_df.head(10000)

In [189]:
song_df['song'] = song_df['title'].map(str) + " _ " + song_df['artist_name']

In [190]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0,The Cove _ Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976,Entre Dos Aguas _ Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007,Stronger _ Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005,Constellations _ Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999,Learn To Fly _ Foo Fighters


In [191]:
 #Most POPULAR Songs in the dataset
song_grouped = song_df.groupby(['song']).agg({'listen_count' : 'count'}).reset_index() 
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = song_grouped['listen_count'].div(grouped_sum)*100 
song_grouped.sort_values(['listen_count', 'song'], ascending = [0,1])

,song,listen_count,percentage
3660,Sehr kosmisch _ Harmonia,45,0.45
4678,Undo _ Björk,32,0.32
5105,You're The One _ Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit) _ Florence + Th...,28,0.28
3655,Secrets _ OneRepublic,28,0.28
...,...,...,...
5139,high fives _ Four Tet,1,0.01
5140,in white rooms _ Booka Shade,1,0.01
5143,paranoid android _ Christopher O'Riley,1,0.01
5149,¿Lo Ves? [Piano Y Voz] _ Alejandro Sanz,1,0.01


In [192]:
users = song_df['user_id'].unique()
len(users)

365

In [194]:
songs = song_df['song'].unique()
len(songs)

5151

In [197]:
train_data, test_data = train_test_split(song_df , test_size = 0.20, random_state = 0)
print(train_data.head(5))

                                       user_id  ...                                   song
7389  94d5bdc37683950e90c56c9b32721edb5d347600  ...          Half Of My Heart _ John Mayer
9275  1012ecfd277b96487ed8357d02fa8326b13696a5  ...  The Beautiful People _ Marilyn Manson
2995  15415fa2745b344bce958967c346f2a89f792f63  ...       Sanctify Yourself _ Simple Minds
5316  ffadf9297a99945c0513cd87939d91d8b602936b  ...       Heart Cooks Brain _ Modest Mouse
356   5a905f000fc1ff3df7ca807d57edb608863db05d  ...                Rorol _ Octopus Project

[5 rows x 8 columns]


In [198]:
 
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id
 
        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
    
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)
 
    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations
    
 


In [199]:
#Class for Item similarity based Recommender System model
class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None
        
    #Get unique items (songs) corresponding to a given user
    def get_user_items(self, user):
        
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        
        return user_items
        
    #Get unique users for a given item (song) 
    # nep: user le suneko item(geet) haru suneka unique users
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())
            
        return item_users # user le suneko geet ko unique listeners haru ko set
        
    #Get unique items (songs) in the training data
    #nep: sabai items (geet) haru
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
            
        return all_items

        #aba hami sange k k cha: user le suneko geet ko aru users ani all songs
        
    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):
            
        ####################################
        #Get users for all songs in user_songs.
        ####################################
        user_songs_users = []        
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))
            
        ###############################################
        #Initialize the item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)
           
        #############################################################
        #Calculate similarity between user songs and all unique songs
        #in the training data
        #############################################################
        for i in range(0,len(all_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())
            
            for j in range(0,len(user_songs)):       
                    
                #Get unique listeners (users) of song (item) j
                users_j = user_songs_users[j]
                    
                #Calculate intersection of listeners of songs i and j
                users_intersection = users_i.intersection(users_j)
                
                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate union of listeners of songs i and j
                    users_union = users_i.union(users_j)
                    
                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0
                    
        
        return cooccurence_matrix
 
    
    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))
        
        #Calculate a weighted average of the scores in cooccurence matrix for all user songs.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()
 
        #Sort the indices of user_sim_scores based upon their value
        #Also maintain the corresponding score
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)
    
        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df = pd.DataFrame(columns=columns)
         
        #Fill the dataframe with top 10 item based recommendations
        rank = 1 
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1
        
        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
 
    #Create the item similarity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id
 
    #Use the item similarity based recommender system model to
    #make recommendations
    def recommend(self, user):
        
        ########################################
        #A. Get all unique songs for this user
        ########################################
        user_songs = self.get_user_items(user)    
            
        print("No. of unique songs for the user: %d" % len(user_songs))
        
        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        ###############################################
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
                
        return df_recommendations
    
    #Get similar items to given items
    def get_similar_items(self, item_list):
        
        user_songs = item_list
        
        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        ###############################################
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        user = ""
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
         
        return df_recommendations

In [200]:
 pm = popularity_recommender_py()

In [201]:
 pm.create(train_data, 'user_id', 'song')

In [202]:
 user_id = users[100]
pm.recommend(user_id)

,user_id,song,score,Rank
3194,b1269307f2ae8c17062c6aea2502b099aad517b6,Sehr kosmisch _ Harmonia,37,1.0
4083,b1269307f2ae8c17062c6aea2502b099aad517b6,Undo _ Björk,27,2.0
931,b1269307f2ae8c17062c6aea2502b099aad517b6,Dog Days Are Over (Radio Edit) _ Florence + Th...,24,3.0
4443,b1269307f2ae8c17062c6aea2502b099aad517b6,You're The One _ Dwight Yoakam,24,4.0
3034,b1269307f2ae8c17062c6aea2502b099aad517b6,Revelry _ Kings Of Leon,21,5.0
3189,b1269307f2ae8c17062c6aea2502b099aad517b6,Secrets _ OneRepublic,21,6.0
4112,b1269307f2ae8c17062c6aea2502b099aad517b6,Use Somebody _ Kings Of Leon,21,7.0
1207,b1269307f2ae8c17062c6aea2502b099aad517b6,Fireflies _ Charttraxx Karaoke,20,8.0
1577,b1269307f2ae8c17062c6aea2502b099aad517b6,Hey_ Soul Sister _ Train,19,9.0
1626,b1269307f2ae8c17062c6aea2502b099aad517b6,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [203]:
is_model = item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'song')

In [204]:
user_id = users[5]
print("%s"%user_id)


4bd88bfb25263a75bbdd467e74018f4ae570e5df


In [205]:
user_items = is_model.get_user_items(users[5])
print("*******************************************************************")
print("-----------------------Trainning data songs for the user userid: %s:" %user_id)
print("******")

*******************************************************************
-----------------------Trainning data songs for the user userid: 4bd88bfb25263a75bbdd467e74018f4ae570e5df:
******


In [206]:
 for user_item in user_items: 
   print(user_item)
   print("*******************************************************************")
   print("Recommendation INPROCESS :")
   print("*************************")

Just Lose It _ Eminem
*******************************************************************
Recommendation INPROCESS :
*************************
Without Me _ Eminem
*******************************************************************
Recommendation INPROCESS :
*************************
16 Candles _ The Crests
*******************************************************************
Recommendation INPROCESS :
*************************
Speechless _ Lady GaGa
*******************************************************************
Recommendation INPROCESS :
*************************
Push It _ Salt-N-Pepa
*******************************************************************
Recommendation INPROCESS :
*************************
Ghosts 'n' Stuff (Original Instrumental Mix) _ Deadmau5
*******************************************************************
Recommendation INPROCESS :
*************************
Say My Name _ Destiny's Child
*******************************************************************
Recommend

In [211]:
#Recommending songs for the user using personalized model
g = is_model.recommend(user_id)
###########################################################################################################################
#Personalized model to make Recommendations for the Following user Id 
user_id = users[7]
user_items = is_model.get_user_items(user_id)

No. of unique songs for the user: 8
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :3429


In [212]:
print("------------------------------------------------------------------------------------")
print("Training data songs for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------
Training data songs for the user userid: 9d6f0ead607ac2a6c2460e4d14fb439a146b7dec:
------------------------------------------------------------------------------------


In [213]:
for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

Swallowed In The Sea _ Coldplay
Life In Technicolor ii _ Coldplay
Life In Technicolor _ Coldplay
The Scientist _ Coldplay
Trouble _ Coldplay
Strawberry Swing _ Coldplay
Lost! _ Coldplay
Clocks _ Coldplay
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------


In [214]:
x = is_model.recommend(user_id)

No. of unique songs for the user: 8
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :3429


In [215]:
song = 'Yellow _ Coldplay'
is_model.get_similar_items([song])

no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :969


,user_id,song,score,rank
0,,Fix You _ Coldplay,0.375000,1
1,,Creep (Explicit) _ Radiohead,0.291667,2
2,,Clocks _ Coldplay,0.280000,3
3,,Seven Nation Army _ The White Stripes,0.250000,4
4,,Paper Planes _ M.I.A.,0.208333,5
5,,Halo _ Beyoncé,0.200000,6
6,,The Funeral (Album Version) _ Band Of Horses,0.181818,7
7,,In My Place _ Coldplay,0.181818,8
8,,Kryptonite _ 3 Doors Down,0.166667,9
9,,When You Were Young _ The Killers,0.166667,10


In [216]:
 import pylab as pl

In [217]:
 #CODE to Plot precision recall curve import pylab as pl
def plot_precision_recall(m1_precision_list, m1_recall_list, m1_label, m2_precision_list, m2_recall_list, m2_label): 
   pl.clf()  
   pl.plot(m1_recall_list, m1_precision_list, label=m1_label)  
   pl.plot(m2_recall_list, m2_precision_list, label=m2_label)  
   pl.xlabel('Recall') 
   pl.ylabel('Precision')
   pl.ylim([0.0, 0.20])    
   pl.xlim([0.0, 0.20])  
   pl.title('Precision-Recall curve') 
   pl.legend(loc = 9, bbox_to_anchor=(0.5, -0.2)) 
   pl.show()

In [218]:
pl.show()